# Citation Sentiment Demo (Offline)

**Target paper:** *Re-emergent tremor of Parkinson's disease* (J Neurol Neurosurg Psychiatry, 1999)

This notebook reproduces the CSV shown in chat using an **offline, self-contained** pipeline:
- five citing papers were preselected (open-access examples)
- one citation sentence was extracted per paper
- five indicators are computed per sentence:
  - `vader_score` (mini VADER-like, 0–1)
  - `textblob_score` (mini polarity, 0–1)
  - `custom_score` (simple lexicon ratio, 0–1)
  - `pos_ratio` (positive tokens / total)
  - `neg_ratio` (negative tokens / total)

> Replace the scoring functions later with real **VADER** / **TextBlob** / transformer models in your online environment.


In [1]:
import re, math
from collections import Counter
from pathlib import Path
import pandas as pd

target_title = "Re-emergent tremor of Parkinson's disease"
target_journal = "J Neurol Neurosurg Psychiatry"

rows = [
    {
        "citing_title": "Re-emergent Tongue Tremor as the Presenting Symptom of Parkinson’s Disease",
        "citing_journal": "Balkan Med J",
        "citing_pmid": "25759787",
        "citation_sentence": "The re-emergent characteristic of the parkinsonian tremor is a key feature which helps to differentiate it from essential tremor."
    },
    {
        "citing_title": "Re-emergent tremor in a patient with Parkinson's disease",
        "citing_journal": "BMJ Case Rep",
        "citing_pmid": "27495177",
        "citation_sentence": "Jankovic et al defined it as re-emergent tremor."
    },
    {
        "citing_title": "Pharmacological Treatment of Tremor in Parkinson’s Disease Revisited",
        "citing_journal": "J Parkinsons Dis",
        "citing_pmid": "36847017",
        "citation_sentence": "This re-emergent tremor is in the same frequency range as resting tremor and the severity of the tremor in the two situations is usually correlated."
    },
    {
        "citing_title": "Passive Monitoring of Parkinson Tremor in Daily Life: A Prototypical Network Approach",
        "citing_journal": "Sensors (Basel)",
        "citing_pmid": "39860736",
        "citation_sentence": "In PD, rest tremor is the most prevalent subtype, although some patients may also experience re-emergent tremor, i.e., a postural tremor with the same frequency as rest tremor."
    },
    {
        "citing_title": "Distinguishing essential tremor from Parkinson’s disease: bedside tests and laboratory evaluations",
        "citing_journal": "Expert Rev Neurother",
        "citing_pmid": "22650171",
        "citation_sentence": "Finally, the postural tremor of PD has been described as a re-emergent tremor, which occurs after a variable latency period after assuming an outstretched posture."
    },
]

POS_WORDS = {
    "key": 1.5, "helps": 1.5, "support": 1.2, "supports": 1.2, "supported": 1.2,
    "consistent": 1.0, "confirm": 1.2, "confirms": 1.2, "confirming": 1.2,
    "correlated": 0.8, "benefit": 1.2, "beneficial": 1.2, "similar": 0.5,
    "novel": 0.8, "improved": 1.2, "improves": 1.2, "improvement": 1.0,
    "evidence": 0.6, "demonstrate": 1.2, "demonstrates": 1.2, "demonstrated": 1.2,
    "relevant": 0.6, "appropriate": 0.6, "useful": 1.0, "important": 0.8,
    "prevalent": 0.2, "characteristic": 0.2, "defined": 0.2
}
NEG_WORDS = {
    "contradict": -1.5, "contradicts": -1.5, "inconsistent": -1.2,
    "however": -0.6, "but": -0.4, "though": -0.3, "nevertheless": -0.3,
    "lack": -1.2, "lacking": -1.2, "insufficient": -1.2, "limited": -0.8,
    "poor": -1.0, "weak": -0.8, "debatable": -0.6, "unclear": -0.5, "not": -0.4,
    "no": -0.4, "without": -0.2
}
BOOSTERS = {"very": 0.3, "highly": 0.3, "strongly": 0.3, "particularly": 0.2, "extremely": 0.4}
DAMPENERS = {"slightly": -0.2, "somewhat": -0.2, "mildly": -0.2, "often": -0.1, "usually": -0.1}
NEGATORS = {"not", "no", "never", "none", "hardly", "scarcely", "without"}

def tokenize(text: str):
    return re.findall(r"[A-Za-z']+", text.lower())

def mini_vader_score(text: str) -> float:
    tokens = tokenize(text)
    score = 0.0
    for i, w in enumerate(tokens):
        val = 0.0
        if w in POS_WORDS:
            val = POS_WORDS[w]
        elif w in NEG_WORDS:
            val = NEG_WORDS[w]
        window = tokens[max(0, i-3):i]
        if val != 0.0 and any(n in window for n in NEGATORS):
            val *= -0.75
        if i > 0:
            prev = tokens[i-1]
            if prev in BOOSTERS:   val *= (1 + BOOSTERS[prev])
            if prev in DAMPENERS:  val *= (1 + DAMPENERS[prev])
        score += val
    norm = max(1.0, len(tokens)) ** 0.5
    compound = math.tanh(score / norm)
    return (compound + 1.0) / 2.0

def simple_textblobish(text: str) -> float:
    tokens = tokenize(text)
    if not tokens: return 0.5
    score = 0.0
    for w in tokens:
        if w in POS_WORDS: score += POS_WORDS[w] * 0.8
        if w in NEG_WORDS: score += NEG_WORDS[w] * 1.1
    polarity = max(-1.0, min(1.0, score / (len(tokens) ** 0.7)))
    return (polarity + 1.0) / 2.0

def simple_pos_neg_ratios(text: str):
    tokens = tokenize(text)
    counts = Counter(tokens)
    pos = sum(counts[w] for w in POS_WORDS if w in counts)
    neg = sum(counts[w] for w in NEG_WORDS if w in counts)
    total = max(len(tokens), 1)
    return pos/total, neg/total, pos, neg, total

def simple_custom_score(text: str) -> float:
    pos_r, neg_r, *_ = simple_pos_neg_ratios(text)
    return (pos_r - neg_r + 1.0) / 2.0

records = []
for r in rows:
    sent = r["citation_sentence"]
    vader = mini_vader_score(sent)
    tb = simple_textblobish(sent)
    pos_r, neg_r, *_ = simple_pos_neg_ratios(sent)
    custom = simple_custom_score(sent)
    records.append({
        "target_title": target_title,
        "target_journal": target_journal,
        "citing_title": r["citing_title"],
        "citing_journal": r["citing_journal"],
        "citing_pmid": r["citing_pmid"],
        "citation_sentence": sent,
        "vader_score": round(vader, 3),
        "textblob_score": round(tb, 3),
        "custom_score": round(custom, 3),
        "pos_ratio": round(pos_r, 3),
        "neg_ratio": round(neg_r, 3),
    })

df = pd.DataFrame.from_records(records, columns=[
    "target_title","target_journal","citing_title","citing_journal","citing_pmid",
    "citation_sentence","vader_score","textblob_score","custom_score","pos_ratio","neg_ratio"
])

out_path = Path("reemergent_tremor_citation_sentiment_demo.csv")
df.to_csv(out_path, index=False, encoding="utf-8")
df

,target_title,target_journal,citing_title,citing_journal,citing_pmid,citation_sentence,vader_score,textblob_score,custom_score,pos_ratio,neg_ratio
0,Re-emergent tremor of Parkinson's disease,J Neurol Neurosurg Psychiatry,Re-emergent Tongue Tremor as the Presenting Sy...,Balkan Med J,25759787,The re-emergent characteristic of the parkinso...,0.807,0.657,0.575,0.150,0.0
1,Re-emergent tremor of Parkinson's disease,J Neurol Neurosurg Psychiatry,Re-emergent tremor in a patient with Parkinson...,BMJ Case Rep,27495177,Jankovic et al defined it as re-emergent tremor.,0.533,0.517,0.556,0.111,0.0
2,Re-emergent tremor of Parkinson's disease,J Neurol Neurosurg Psychiatry,Pharmacological Treatment of Tremor in Parkins...,J Parkinsons Dis,36847017,This re-emergent tremor is in the same frequen...,0.570,0.533,0.519,0.038,0.0
3,Re-emergent tremor of Parkinson's disease,J Neurol Neurosurg Psychiatry,Passive Monitoring of Parkinson Tremor in Dail...,Sensors (Basel),39860736,"In PD, rest tremor is the most prevalent subty...",0.518,0.507,0.517,0.033,0.0
4,Re-emergent tremor of Parkinson's disease,J Neurol Neurosurg Psychiatry,Distinguishing essential tremor from Parkinson...,Expert Rev Neurother,22650171,"Finally, the postural tremor of PD has been de...",0.500,0.500,0.500,0.000,0.0
